# Trabalho de Bioinformática
- Ana Cristina Silva de Oliveira, 11965630
- Fernando Henrique Paes Generich, 11795342
- Vítor Amorim Fróis, 12543440

In [1]:
import pandas as pd
from Bio import Entrez, SeqIO, Seq
from tqdm import tqdm

## Leitura dos arquivos .gff3
Vamos selecionar todas as linhas em que a Strand é positiva:

In [35]:
filenames = [
  'TEAnnotationFinal_Helitron.gff3',
  'TEAnnotationFinal_LINE.gff3',
  'TEAnnotationFinal_LTR.gff3',
  'TEAnnotationFinal_MITE.gff3',
  'TEAnnotationFinal_SINE.gff3',
  'TEAnnotationFinal_TIR.gff3'
]

def process_file(filename: str) -> pd.DataFrame:
  df = pd.read_csv(f'data/{filename}', sep='\t', header=None)
  df.drop(df[ df[6] != '+' ].index , inplace = True)
  df = df.drop( [1, 5, 7, 8], axis=1)
  return df

te_df = pd.DataFrame()

# Juntando os dados em um único DataFrame
for f in filenames:
  te_df = pd.concat([te_df, process_file(f)], ignore_index=True)

# Renomeando colunas
te_df.rename(columns={0 : "Chr", 2: "Class", 3: "Start", 4: "End", 6: "Strand"}, inplace = True)

# Seleciona Chr numérico apenas
te_df = te_df[te_df['Chr'].astype(str).str.isdigit()]

te_df.head()

,Chr,Class,Start,End,Strand
0,7,Class II subclass 2/Helitron/Helitron,82856122,82857584,+
1,1,Class II subclass 2/Helitron/Helitron,239302471,239302834,+
2,8,Class II subclass 2/Helitron/Helitron,2261318,2261604,+
3,5,Class II subclass 2/Helitron/Helitron,124665404,124665608,+
4,9,Class II subclass 2/Helitron/Helitron,77998293,78002783,+


## Obtendo as sequências de cromossomos do NCBI
No total, são 10 cromossomos

In [5]:
Entrez.email = "fernando_gene@usp.br"

ind = 618874

allchromosomes = []

for i in tqdm(range(10)):
  gen_bank_term = "LR" + str(ind+i) + ".1"

  handle = Entrez.esearch(db="nucleotide", term=gen_bank_term, retmax="10")
  rec_list = Entrez.read(handle)
  handle.close()

  id_list = rec_list['IdList']
  handle = Entrez.efetch(db='nucleotide', id=id_list, rettype='fasta', retmode="text")
  recs = list(SeqIO.parse(handle, 'fasta'))
  handle.close()

  allchromosomes.append(recs[0])

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [09:06<00:00, 54.64s/it]


In [16]:
chromosome_dict = {
  'Number': [],
  'Sequence': []
}

for i in allchromosomes:
  chromosome_dict['Number'].append(i.id)
  chromosome_dict['Sequence'].append(i.seq)

chromosome_df = pd.DataFrame(chromosome_dict)

chromosome_df

,Number,Sequence
0,LR618874.1,"(T, C, A, T, G, G, C, T, A, T, T, T, T, C, A, ..."
1,LR618875.1,"(T, T, C, T, C, A, T, T, A, T, A, T, C, C, T, ..."
2,LR618876.1,"(C, C, T, A, A, A, C, C, C, T, A, A, A, C, C, ..."
3,LR618877.1,"(C, C, T, A, A, A, C, C, C, T, A, A, A, C, C, ..."
4,LR618878.1,"(C, T, A, A, A, C, C, T, A, A, A, C, A, T, C, ..."
5,LR618879.1,"(A, A, A, A, C, C, C, T, A, A, A, C, C, C, T, ..."
6,LR618880.1,"(C, T, A, A, A, A, C, C, C, T, A, A, A, C, C, ..."
7,LR618881.1,"(A, A, A, C, C, T, A, A, A, C, C, C, T, A, A, ..."
8,LR618882.1,"(G, T, C, G, C, T, C, A, T, G, G, C, T, A, T, ..."
9,LR618883.1,"(T, A, A, A, C, C, C, T, A, A, A, C, C, C, T, ..."


## Efetuando um Join entre os DataFrames 

In [2]:
def get_sequence(chromosome: int, start: int, end: int) -> Seq.Seq:
  return chromosome_df.Sequence[chromosome-1][start:end+1]

te_df['Sequence'] = te_df.apply(lambda x: get_sequence(int(x.Chr), int(x.Start), int(x.End)), axis=1)

NameError: name 'te_df' is not defined

Deleta linhas que possuem NaN

In [10]:
te_df = te_df.dropna()
te_df.head()

,Chr,Class,Start,End,Strand,Sequence
0,7,Class II subclass 2/Helitron/Helitron,82856122,82857584,+,AGCTTCGTCACCAGCTTTGCTCCGACCACCCTTTGTCCATACTAAC...
1,1,Class II subclass 2/Helitron/Helitron,239302471,239302834,+,TCAGGGTTGCTTCTTGGCGAAGACAGGGCCTCGGGCGAGCCAGAAA...
2,8,Class II subclass 2/Helitron/Helitron,2261318,2261604,+,CGCCCAAGCAGACGGTCACCATCAGCGAAGACCTCACTTCGCATGA...
3,5,Class II subclass 2/Helitron/Helitron,124665404,124665608,+,ATGCCAAGTCGTGTCAAACGACTTAGGGTAGGGGTCAACTTTCTCC...
4,9,Class II subclass 2/Helitron/Helitron,77998293,78002783,+,TTAGGTTATTTATATACTAGTTTATGTTGATGATATAATCATCACT...


Salva o Dataframe como `.csv`

In [ ]:
te_df.to_csv('data/transposable_elements.csv')

Leitura do arquivo csv

In [3]:
te_df = pd.read_csv('data/tranposable_elements.csv', index_col=0)

In [4]:
te_df

,Chr,Class,Start,End,Strand,Sequence
0,7,Class II subclass 2/Helitron/Helitron,82856122,82857584,+,AGCTTCGTCACCAGCTTTGCTCCGACCACCCTTTGTCCATACTAAC...
1,1,Class II subclass 2/Helitron/Helitron,239302471,239302834,+,TCAGGGTTGCTTCTTGGCGAAGACAGGGCCTCGGGCGAGCCAGAAA...
2,8,Class II subclass 2/Helitron/Helitron,2261318,2261604,+,CGCCCAAGCAGACGGTCACCATCAGCGAAGACCTCACTTCGCATGA...
3,5,Class II subclass 2/Helitron/Helitron,124665404,124665608,+,ATGCCAAGTCGTGTCAAACGACTTAGGGTAGGGGTCAACTTTCTCC...
4,9,Class II subclass 2/Helitron/Helitron,77998293,78002783,+,TTAGGTTATTTATATACTAGTTTATGTTGATGATATAATCATCACT...
...,...,...,...,...,...,...
341707,2,Class II subclass 1/TIR/Ginger,84750621,84751004,+,CAGGAGATATATTGAGCACTGCTCTTTCAAGAAGGTCAACCGAAGG...
341708,1,Class II subclass 1/TIR/Ginger,223982226,223983233,+,TCCCTCATGCCAATGAAGGCACACACAGTACTAAAGGCCTTCTGAC...
341709,4,Class II subclass 1/TIR/Ginger,7068255,7069168,+,TATCTCAGAGCCAATGCTTTTCACCTCATGTTTTACCAACTCAGTC...
341710,4,Class II subclass 1/TIR/Ginger,13348879,13349600,+,TGGCGTATTGTCGCGATATGTATTCATTTAATTTGTAGCTCTTTCT...


## Classificação com Kernels

In [112]:
from Levenshtein import distance as levenshtein_distance
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [219]:
small_df = te_df.dropna().sample(1000).reset_index(drop=True)

In [215]:
def edit_distance_kernel(A, B):
    kernel_matrix = np.zeros((len(A), len(B)))
    for a in A:
        for b in B:
            i = int(a[0])
            j = int(b[0])
            kernel_matrix[i, j] = levenshtein_distance(
                small_df.iloc[i]['Sequence'], 
                small_df.iloc[j]['Sequence']
            )
    return np.exp(-kernel_matrix/normalizing_term)

### Split entre treino e teste

In [256]:
X = list(small_df['Sequence'])
y = np.array(small_df['Chr']).reshape(-1, 1)

le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y)

/home/vitor/Documents/TransposableElements/venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Criação da matriz kernel

In [310]:
from strkernel.mismatch_kernel import MismatchKernel
from strkernel.mismatch_kernel import preprocess

K_train = MismatchKernel(l=4, k=5, m=1).get_kernel(preprocess(X_train)).kernel
K_train = np.exp(K_train)

### Treinamento

In [312]:
classifier = SVC(kernel='precomputed')
classifier.fit(K_train, y_train)

SVC(kernel='precomputed')

In [313]:
y_pred = classifier.predict(K_train)

### Avaliação do modelo no conjunto de treinamento

In [314]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_train)

0.8973333333333333